In [1]:
import subprocess
import shutil
import torch
import torchvision
from torchvision import transforms
from torchsummary import summary

/usr/local/lib/python3.6/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1))
])

train_data = torchvision.datasets.ImageFolder(root='dataset/train', transform=transform)
test_data = torchvision.datasets.ImageFolder(root='dataset/test', transform=transform)


In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=32, shuffle=True, num_workers=6)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=32, shuffle=True, num_workers=4)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.AvgPool1d(4, 4)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.AvgPool2d(4, 4)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.pool = nn.AvgPool2d(4, 4)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.pool = nn.AvgPool2d(4, 4)
        self.fc = nn.Linear(64, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc(x)
        return x


model = Net()
model = model.to(device)

In [7]:
summary(model, (3, 640, 640))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 638, 638]             896
         AvgPool2d-2         [-1, 32, 159, 159]               0
            Conv2d-3         [-1, 64, 157, 157]          18,496
         AvgPool2d-4           [-1, 64, 39, 39]               0
            Conv2d-5           [-1, 64, 37, 37]          36,928
         AvgPool2d-6             [-1, 64, 9, 9]               0
            Conv2d-7             [-1, 64, 7, 7]          36,928
         AvgPool2d-8             [-1, 64, 1, 1]               0
            Linear-9                    [-1, 4]             260
Total params: 93,508
Trainable params: 93,508
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 4.69
Forward/backward pass size (MB): 119.06
Params size (MB): 0.36
Estimated Total Size (MB): 124.10
-----------------------------------------

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [9]:
early_batches = {2, 4, 6, 8, 10, 15, 20, 30, 50,
                                75, 100, 150, 200, 300, 400, 500,
                               600, 700, 800}

In [ ]:
test_accs = dict()

for epoch in range(20):  # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if epoch == 0 and i in early_batches:
            torch.save(model.state_dict(), f'saved_models/smallboy/epoch_{epoch}_batch_{i}.pth')
        if i % 250 == 0:
            torch.save(model.state_dict(), f'saved_models/smallboy/epoch_{epoch}_batch_{i}.pth')

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0
        if (i % 250 == 1) or (epoch == 0 and i in early_batches):
            correct = 0
            total = 0
            with torch.no_grad():
                for ix, data in enumerate(test_loader):
                    if ix > 100:
                        break
                    images, labels = data
                    images, labels = images.cuda(), labels.cuda()
                    # calculate outputs by running images through the network
                    outputs = model(images)
                    # the class with the highest energy is what we choose as prediction
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            acc = correct / total
            test_accs[(epoch, i)] = acc
            print(f'Accuracy of the network on the test images: {100*acc} %')    
            
with open('accuracies_smallboi.json', 'w') as fp:
    json.dump(test_accs, fp)
print('Finished Training')

Accuracy of the network on the test images: 19.801980198019802 %
Accuracy of the network on the test images: 19.43069306930693 %
Accuracy of the network on the test images: 26.175742574257427 %
Accuracy of the network on the test images: 24.53589108910891 %
Accuracy of the network on the test images: 26.268564356435643 %
Accuracy of the network on the test images: 26.794554455445546 %
Accuracy of the network on the test images: 26.144801980198018 %
Accuracy of the network on the test images: 27.413366336633665 %
Accuracy of the network on the test images: 27.846534653465348 %
Accuracy of the network on the test images: 32.42574257425743 %
Accuracy of the network on the test images: 33.415841584158414 %
[1,   100] loss: 1.366
Accuracy of the network on the test images: 40.19183168316832 %
Accuracy of the network on the test images: 40.129950495049506 %
[1,   200] loss: 1.278
Accuracy of the network on the test images: 44.36881188118812 %
Accuracy of the network on the test images: 46.72

[1, 14300] loss: 0.490
[1, 14400] loss: 0.486
[1, 14500] loss: 0.499
Accuracy of the network on the test images: 80.22896039603961 %
[1, 14600] loss: 0.473
[1, 14700] loss: 0.485
Accuracy of the network on the test images: 80.04331683168317 %
[1, 14800] loss: 0.457
[1, 14900] loss: 0.459
Accuracy of the network on the test images: 80.87871287128714 %
[2,   100] loss: 0.479
[2,   200] loss: 0.460
Accuracy of the network on the test images: 82.20915841584159 %
[2,   300] loss: 0.491
[2,   400] loss: 0.450
[2,   500] loss: 0.475
Accuracy of the network on the test images: 79.67202970297029 %
[2,   600] loss: 0.477
[2,   700] loss: 0.462
Accuracy of the network on the test images: 81.34282178217822 %
[2,   800] loss: 0.472
[2,   900] loss: 0.458
[2,  1000] loss: 0.485
Accuracy of the network on the test images: 80.6930693069307 %
[2,  1100] loss: 0.459
[2,  1200] loss: 0.442
Accuracy of the network on the test images: 79.36262376237624 %
[2,  1300] loss: 0.461
[2,  1400] loss: 0.473
[2,  1

In [ ]:
"""correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        classes = classes.cuda()
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for t, p in zip(classes.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
print(confusion_matrix.diag()/confusion_matrix.sum(1))
"""